## Imports and Defs

### Imports

In [1]:
import requests
from bs4 import BeautifulSoup
import zipfile
import os
import io

from urllib.parse import urljoin
from io import BytesIO
import pandas as pd
import numpy as np
pd.set_option('display.max_colwidth', None)
import copy
import unidecode
import string
from tqdm import tqdm

import time
import datetime
import pickle


In [2]:
import assets.helper as b3

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys

from webdriver_manager.chrome import ChromeDriverManager
import unidecode
import string

import os
import re

import pandas as pd
import numpy as np

from google.cloud import storage
import io
from collections import OrderedDict

import requests
from bs4 import BeautifulSoup
import unidecode
import string
import datetime
import time

import pickle
from urllib.parse import urljoin
import zipfile
from lxml import html

from tqdm import tqdm
import shutil


In [3]:
# selenium functions
def load_browser():
    """
    Launches chromedriver and creates a wait object.
    
    Returns:
    tuple: A tuple containing a WebDriver instance and a WebDriverWait instance.
    """
    # Define the options for the ChromeDriver.
    options = Options()
    options.add_argument('--headless')  # Run in headless mode.
    options.add_argument('--no-sandbox')  # Avoid sandboxing.
    options.add_argument('--disable-dev-shm-usage')  # Disable shared memory usage.
    options.add_argument('--disable-blink-features=AutomationControlled')  # Disable automated control.
    # options = Options()
    options.add_argument('start-maximized')  # Maximize the window on startup.

    # Install and start the ChromeDriver service, passing in the options.
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
    
    # Define the exceptions to ignore during WebDriverWait.
    exceptions_ignore = (NoSuchElementException, StaleElementReferenceException)
    
    # Create a WebDriverWait instance for the driver, using the specified wait time and exceptions to ignore.
    wait = WebDriverWait(driver, driver_wait_time, ignored_exceptions=exceptions_ignore)
    set_driver_and_wait(driver, wait)

    # Return a tuple containing the driver and the wait object.
    return driver, wait



### Variables and Functions

In [4]:
import assets.helper as b3
import assets.functions as run


## Content

#### Get Classificacao Setorial

In [ ]:
url = 'https://sistemaswebb3-listados.b3.com.br/listedCompaniesPage/classification?language=pt-br'
max_retries = 5
sleep_time = 0.1
setorial = []

# Navigate to the specified URL
driver, wait = run.load_browser()


In [ ]:
driver.get(url)

# Locate the select element for sector dropdown
select_element_xpath = '//*[@id="divContainerIframeB3"]/div/div/app-companies-home-filter-classification/form/div[1]/div/div/select'
dropdown = wait.until(EC.presence_of_element_located((By.XPATH, select_element_xpath)))
select = Select(dropdown)

idx = 0
while idx < len(select.options):
    for attempt in range(max_retries):
        try:
            # Re-fetch the dropdown
            dropdown = wait.until(EC.presence_of_element_located((By.XPATH, select_element_xpath)))
            select = Select(dropdown)
            
            # Re-fetch the specific option
            option = select.options[idx]
            option_value = option.get_attribute("value")
            setor = run.clean_text(option_value)
            print(idx, setor)
        
            # Select the option
            select.select_by_value(option_value)

            print(idx, setor)
        
            # Select the option
            select.select_by_value(option_value)

            # get segmentos
            setorial = []
            max_retries = 5
            sleep_time = 0.1

            # Store the current URL before processing any segment
            current_url = driver.current_url

            # Extract table data
            table_xpath = '//*[@id="divContainerIframeB3"]/div/div/app-companies-home-filter-classification/form/table'
            table_loaded = wait.until(EC.presence_of_element_located((By.XPATH, table_xpath)))
            rows = wait.until(EC.presence_of_all_elements_located((By.XPATH, f"{table_xpath}/tbody/tr")))

            for idx_sub, _ in enumerate(rows):
                # time.sleep(1)
                # Re-locate rows each time to avoid stale element exceptions
                rows_fresh = wait.until(EC.presence_of_all_elements_located((By.XPATH, f"{table_xpath}/tbody/tr")))
                row = rows_fresh[idx_sub]

                subsetor = row.find_element(By.XPATH, "td[1]").text
                subsetor = run.clean_text(subsetor)

                try:
                    segmentos = row.find_elements(By.XPATH, "td[2]/p/a")
                except Exception as e:
                    segmentos = wait.until(EC.presence_of_all_elements_located((By.XPATH, "td[2]/p/a")))
                
                for idx_seg, _ in enumerate(segmentos):
                    # Re-fetch rows and segment links
                    rows_fresh = wait.until(EC.presence_of_all_elements_located((By.XPATH, f"{table_xpath}/tbody/tr")))
                    row = rows_fresh[idx_sub]
                    try:
                        segmentos_fresh = row.find_elements(By.XPATH, "td[2]/p/a")
                    except Exception as e:
                        segmentos_fresh = wait.until(EC.presence_of_all_elements_located((By.XPATH, "td[2]/p/a")))
                    item = segmentos_fresh[idx_seg]
                    segmento = run.clean_text(item.text)
                    print(f'{idx} {setor} -> {idx_sub} {subsetor} -> {idx_seg} {segmento}')


            idx += 1  # Only increase the index after successful processing
            break  # Break out of the retry loop after success

        except Exception as e:
            driver.refresh()  # Refresh the page
            print('o' * (attempt + 1) + 'ps..', e)
            time.sleep(sleep_time)


In [ ]:
driver.get(url)

# Locate the select element for sector dropdown
select_element_xpath = '//*[@id="divContainerIframeB3"]/div/div/app-companies-home-filter-classification/form/div[1]/div/div/select'
dropdown = wait.until(EC.presence_of_element_located((By.XPATH, select_element_xpath)))
select = Select(dropdown)
# Cycle through each option in the dropdown
for idx in range(len(select.options)):
    for attempt in range(max_retries):
        try:
            # Re-fetch the dropdown
            dropdown = wait.until(EC.presence_of_element_located((By.XPATH, select_element_xpath)))
            select = Select(dropdown)
            
            # Re-fetch the specific option
            option = select.options[idx]
            option_value = option.get_attribute("value")
            setor = run.clean_text(option_value)
            print(idx, setor)
        
            # Select the option
            select.select_by_value(option_value)

            # get segmentos
            setorial = []
            max_retries = 5
            sleep_time = 0.1

            # Store the current URL before processing any segment
            current_url = driver.current_url

            # Extract table data
            table_xpath = '//*[@id="divContainerIframeB3"]/div/div/app-companies-home-filter-classification/form/table'
            rows = wait.until(EC.presence_of_all_elements_located((By.XPATH, f"{table_xpath}/tbody/tr")))

            for idx_sub, _ in enumerate(rows):
                # time.sleep(1)
                # Re-locate rows each time to avoid stale element exceptions
                rows_fresh = wait.until(EC.presence_of_all_elements_located((By.XPATH, f"{table_xpath}/tbody/tr")))
                row = rows_fresh[idx_sub]

                subsetor = row.find_element(By.XPATH, "td[1]").text
                subsetor = run.clean_text(subsetor)

                try:
                    segmentos = row.find_elements(By.XPATH, "td[2]/p/a")
                except Exception as e:
                    segmentos = wait.until(EC.presence_of_all_elements_located((By.XPATH, "td[2]/p/a")))
                
                for idx_seg, _ in enumerate(segmentos):
                    # Re-fetch rows and segment links
                    rows_fresh = wait.until(EC.presence_of_all_elements_located((By.XPATH, f"{table_xpath}/tbody/tr")))
                    row = rows_fresh[idx_sub]
                    try:
                        segmentos_fresh = row.find_elements(By.XPATH, "td[2]/p/a")
                    except Exception as e:
                        segmentos_fresh = wait.until(EC.presence_of_all_elements_located((By.XPATH, "td[2]/p/a")))
                    item = segmentos_fresh[idx_seg]
                    segmento = run.clean_text(item.text)
                    print(f'{idx} {setor} -> {idx_sub} {subsetor} -> {idx_seg} {segmento}')

        except Exception as e:
            driver.refresh()  # Refresh the page
            print('o' * (attempt + 1) + 'ps..', e)
            time.sleep(sleep_time)
            break


#### Merge Mix and Match all

In [27]:
try:
    math = run.load_pkl(f'{b3.app_folder}math')
except Exception as e:
    math = run.get_math()
    math = run.save_pkl(math, f'{b3.app_folder}math')


In [ ]:

try:
    setorial = run.load_pkl(f'{b3.app_folder}setorial')
except Exception as e:
    setorial = run.get_classificacao_setorial(setorial='')
    setorial = run.save_pkl(setorial, f'{b3.app_folder}setorial')
# setorial.head(3)

In [5]:

try:
    company_b3 = run.load_pkl(f'{b3.app_folder}company_b3')
except Exception as e:
    company_b3 = run.b3_grab(b3.search_url)
    company_b3 = run.save_pkl(company_b3, f'{b3.app_folder}company_b3')


In [ ]:

try:
    company = run.load_pkl(f'{b3.app_folder}company')
except Exception as e:
    company = run.get_companies(math, setorial)
    company = run.save_pkl(company, f'{b3.app_folder}company')


In [ ]:
# SETOR: O setor econômico mais amplo ao qual a empresa pertence.
# SUBSETOR: Uma categorização mais específica dentro do setor mais amplo.
# SEGMENTO: Uma classificação ainda mais granular do negócio da empresa.
# DENOM_CIA: Esta é a denominação ou nome da empresa.
# COMPANHIA: Nome ou denominação oficial da empresa listada.
# PREGAO: Refere-se ao nome pelo qual a empresa é conhecida no pregão da bolsa de valores.
# LISTAGEM: Categoria ou segmento de listagem da empresa na bolsa de valores, que pode indicar o nível de governança corporativa ou outros critérios.
# TICK: Abreviação ou símbolo da empresa usada no mercado de ações.
# TICKERS: Símbolos de negociação da empresa em diferentes mercados ou plataformas.
# CD_CVM: Este poderia ser um código ou identificador único relacionado à empresa, possivelmente relacionado à Comissão de Valores Mobiliários do Brasil (CVM).
# CVM: Código ou identificador relacionado à empresa na Comissão de Valores Mobiliários, o órgão regulador do mercado de capitais no Brasil.
# ISIN: Número de Identificação Internacional de Valores Mobiliários – um identificador único para valores mobiliários.
# CNPJ_CIA: Este é o número do Cadastro Nacional da Pessoa Jurídica (CNPJ) da empresa, um identificador único para empresas no Brasil.
# CNPJ: Cadastro Nacional da Pessoa Jurídica – é o número de identificação das empresas brasileiras.
# SITE: Site oficial ou página relevante da empresa.
# ATIVIDADE: Descreve a principal atividade de negócios da empresa.

# ANO: Este é o ano ao qual os dados se referem.
# DT_REFER: Esta é a data de referência para a entrada de dados.
# DT_FIM_EXERC: Esta é a data final para o exercício ou período de relato financeiro.
# DT_INI_EXERC: Esta poderia ser a data inicial para o exercício ou período de relato financeiro.

# AGRUPAMENTO: Isso descreve o nível de agregação dos dados. Por exemplo, 'con' pode indicar dados consolidados.
# BALANCE_SHEET: Isso indica a seção específica da demonstração financeira, como Balanço Patrimonial ('BPA').
# GRUPO_DFP: Isso representa o tipo de grupo de demonstração financeira. Por exemplo, 'DF Consolidado - Balanço Patrimonial Ativo' sugere que é um balanço patrimonial consolidado focado em ativos.
# CD_CONTA: Este poderia ser um código ou identificador único relacionado a uma conta específica ou item de linha na demonstração financeira.
# DS_CONTA: Descreve a conta específica ou item de linha na demonstração financeira, como 'Ativo Total'.

# VL_CONTA: Representa o valor associado à conta específica ou item de linha.
# MOEDA: Isso indica a moeda na qual os valores são representados. 'REAL' sugere Real Brasileiro.
# ESCALA_MOEDA: Isso fornece a escala ou unidade para os valores monetários. 'MIL' pode indicar que os valores estão em milhares.

# ST_CONTA_FIXA: Pode indicar o status ou tipo de conta. O significado de valores como 'S' dependeria do contexto dos dados.
# COLUNA_DF: O propósito desta coluna não é imediatamente claro a partir da amostra. Pode representar algum tipo de classificação ou categorização relacionada aos dados financeiros.

# ESCRITURADOR: Entidade ou empresa responsável por registrar ou gerenciar os valores mobiliários da empresa.
# ACIONISTAS: Informações ou identificadores relacionados aos acionistas da empresa.

# FILENAME: Este é o arquivo de onde os dados são originados. Ele fornece o nome do arquivo que contém a respectiva entrada de dados.
# DEMONSTRATIVO: Este representa o tipo de demonstração financeira. Pode indicar se os dados são de um relatório intermediário (como 'itr') ou de outro tipo de relatório financeiro.
# VERSAO: Isso pode representar a versão ou iteração dos dados/relatórios financeiros.

columns = [
    'SETOR_x',
    'SUBSETOR_x',
    'SEGMENTO_x',
    'DENOM_CIA',
        # 'COMPANHIA',
    'PREGAO',
    'LISTAGEM',
    'TICK',
    'TICKERS',
    'CD_CVM',
        # 'CVM',
        # 'ISIN',
    'CNPJ_CIA',
        # 'CNPJ',
    'SITE',
    'ATIVIDADE',
        # 'ANO',
    'DT_REFER',
        # 'DT_FIM_EXERC',
        # 'DT_INI_EXERC',
    'AGRUPAMENTO',
    'BALANCE_SHEET',
    # 'GRUPO_DFP',
    'CD_CONTA',
    'DS_CONTA',
    'VL_CONTA',
    # 'MOEDA',
    # 'ESCALA_MOEDA',
    # 'ST_CONTA_FIXA',
    # 'COLUNA_DF',
    'ESCRITURADOR',
    'ACIONISTAS', 
    # 'FILENAME', 
    # 'DEMONSTRATIVO', 
    # 'VERSAO',
]


In [11]:
company_b3 = run.load_pkl(f'{b3.app_folder}company_b3')


In [66]:
# Initialize a new dictionary to hold the results
bovespa = {}

# Iterate through each year's dataframe in the math dictionary
for year, df in math.items():
    print(year)
    # Merge the dataframe for that year with company_b3 based on CNPJ
    merged_df = pd.merge(df, company_b3, how='right', left_on='CNPJ_CIA', right_on='CNPJ')
    
    # Group by 'SETOR' and store each group's dataframe in the new dictionary
    for sector, sector_df in merged_df.groupby('SETOR'):
        # If the sector is already a key in the dictionary, append the new dataframe to it
        if sector in bovespa:
            bovespa[sector] = pd.concat([bovespa[sector], sector_df], ignore_index=True)
        else:
            bovespa[sector] = sector_df

# Now, merged_dict has sectors as keys and the corresponding dataframes as values


2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
